In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

In [3]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [4]:
simp = PlaintextCorpusReader(datapath+'/Corpora/wiki/enwiki_20200520/','enwiki_20200520_10pc.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [5]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [6]:
# Load n-gram dataframe produced in word2vec iteration

ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/10pc_ngram_eval.pkl')

In [7]:
ngram_eval

,ngram,poisson,len,batch
0,"(of, the)",3.874652e+06,2,-2
1,"(References, External, links)",2.566994e+06,3,1
2,"(External, links)",2.229096e+06,2,2
3,"(in, the)",2.094387e+06,2,-2
4,"(0, 0, 0)",1.798530e+06,3,-2
...,...,...,...,...
499995,"(Work, started, on)",3.883693e+02,3,8
499996,"(of, these, techniques)",3.883679e+02,3,8
499997,"(Stadio, Flaminio)",3.883678e+02,2,1
499998,"(IRE, 3, C)",3.883678e+02,3,1


In [8]:
eval_count = 500000
min_freq = 20

In [9]:
ngram_eval.batch.value_counts()

 1     94011
 2     79538
 3     69221
 4     56154
 5     48934
 6     38288
 7     30374
-1     29076
 8     22495
 9     19272
 10    11300
-2      1337
Name: batch, dtype: int64

In [10]:
from redis import Redis
from rediscollections import RedisHashMap, key_val_tuples_hash

In [11]:
# No. of simplex words to include in vocabulary (plus n-grams for each batch)
simplex_vocab_count = 300000

batch_count = max(ngram_eval.batch)
batch_count

10

In [12]:
# Connect to Redis server for unigram stats

r = Redis()

w10p_rhm = RedisHashMap(r, 'w10p:w10p_ug')

In [13]:
print(w10p_rhm[('April',)])

b'171132'


In [14]:
from heapq import nlargest
from operator import itemgetter
import pickle

In [15]:
def vocab_gen(rhm):
    for k, v in key_val_tuples_hash(rhm):
        yield (k[0], int(v))

In [16]:
simplex_vocab = set(k for k,_ in nlargest(simplex_vocab_count, vocab_gen(w10p_rhm), key=itemgetter(1)) )

In [17]:
'A' in simplex_vocab

True

In [18]:
len(simplex_vocab)

300000

In [19]:
# Iterate over batches
for bb in range(batch_count):

    this_batch = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    #  Append n-grams to simplex_vocab
    ng_list = [ '+'.join(ng) for ng in this_batch.ngram ]
    
    batch_vocab = simplex_vocab.copy()
    batch_vocab.update(ng_list)

    #  Pickle and save
    with open(datapath+'/Corpora/wiki/enwiki_20200520/10pc_b{}vocab.pkl'.format(bb+1), 'wb') as pfile:
        pickle.dump(batch_vocab, pfile)

In [20]:
ngram_eval[ngram_eval.batch == 10]

,ngram,poisson,len,batch
2534,"(to, ensure, that)",27179.913389,3,10
2772,"(as, early, as)",25348.445486,3,10
4108,"(living, with)",19408.720264,2,10
4554,"(to, the, National)",18013.070615,3,10
5267,"(to, be, built)",16171.022457,3,10
...,...,...,...,...
499825,"(2014, Party, Candidate)",388.480030,3,10
499853,"(Normal, Indirect)",388.463362,2,10
499876,"(on, Larry, King)",388.445246,3,10
499893,"(on, clay)",388.432754,2,10


In [21]:
with open(datapath+'/Corpora/wiki/enwiki_20200520/10pc_b10vocab.pkl', 'rb') as pfile:
    batch_vocab = pickle.load(pfile)
    print('2014+Party+Candidate' in batch_vocab)

True
